In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation
import xgboost as xgb
import sklearn

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

labels = train['TARGET']
train.drop(['TARGET', 'ID'], axis=1, inplace=1)
test.drop(['ID'], axis=1, inplace=1)

In [3]:
unique_values = np.array([train[col].nunique() for col in train])
test.drop(train.columns[unique_values == 1], axis=1, inplace=1)
train.drop(train.columns[unique_values == 1], axis=1, inplace=1)

In [4]:
from itertools import combinations

features_duplicates = set()
visited_cols = set()

for col1 in train:
    visited_cols.add(col1)
    for col2 in train:
        if col2 in visited_cols or col2 in features_duplicates:
            continue
        if np.array_equal(train[col1], train[col2]):
            features_duplicates.add(col2)
            
print 'Дублирующихся признаков: {0}'.format(len(features_duplicates))

Дублирующихся признаков: 29


In [5]:
train.drop(features_duplicates, axis=1, inplace=1)
test.drop(features_duplicates, axis=1, inplace=1)

In [6]:
train['var38mc'] = np.isclose(train.var38, 117310.979016)
train['logvar38'] = train.loc[~train['var38mc'], 'var38'].map(np.log)
train.loc[train['var38mc'], 'logvar38'] = 0

In [7]:
test['var38mc'] = np.isclose(test.var38, 117310.979016)
test['logvar38'] = test.loc[~test['var38mc'], 'var38'].map(np.log)
test.loc[test['var38mc'], 'logvar38'] = 0

In [8]:
train.rename(columns={'var15': 'age'}, inplace=True)
test.rename(columns={'var15': 'age'}, inplace=True)

In [9]:
train = train.replace(-999999,2)
train.loc[train.var3==-999999].shape

(0, 308)

In [10]:
test = test.replace(-999999,2)
test.loc[test.var3==-999999].shape

(0, 308)

In [12]:
train['n0'] = (train==0).sum(axis=1)
test['n0'] = (test==0).sum(axis=1)

In [13]:
train.shape

(76020, 309)

In [14]:
test.shape

(75818, 309)

In [15]:
train[:3]

,var3,age,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,var38mc,logvar38,n0
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,False,10.576564,293
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,False,10.805234,267
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,False,11.117417,278
